Knowledge Distillation 
===============================

**Author**: [Clara Martinez](https://github.com/moonblume/LIVIA.git)


Knowledge distillation is a technique that enables knowledge transfer
from large, computationally expensive models to smaller ones without
losing validity. This allows for deployment on less powerful hardware,
making evaluation faster and more efficient.

Librairies
================


In [66]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from scipy.signal import savgol_filter

from typing import List, Union, Tuple, Any
import statistics

# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading dataset
================

Fisrt, I focus on the physiological signals of the Biovid dataset. In one sample, we have access to 6 classes associated with 0 to 4 pain levels :  

Time: This could be the timestamp or time index when the signal was recorded.

GSR (Galvanic Skin Response): A measure of the electrical conductance of the skin, which varies with the moisture level of the skin. It's often associated with emotional arousal.

ECG (Electrocardiogram): A recording of the electrical activity of the heart over time. It typically consists of waves representing the depolarization and repolarization of the heart muscle during each heartbeat.

EMG (Electromyography) - Trapezius: Measures the electrical activity produced by skeletal muscles. The trapezius muscle is a large superficial muscle that extends longitudinally from the occipital bone to the lower thoracic vertebrae and laterally to the spine of the scapula.

EMG - Corrugator: Electromyography signal from the corrugator supercilii muscle, which is a small facial muscle involved in frowning and expressing negative emotions.

EMG - Zygomaticus: Electromyography signal from the zygomaticus major muscle, which is involved in smiling and expressing positive emotions.  
    
    
Our objective is to predict the pain level of input signals. One signal corresponds to one csv file.



In [67]:
# Define the directory containing the CSV files
biosignals_path = '/home/ens/AU59350/LIVIA/physio/physio_organised/'

# Initialize an empty list to store data for DataFrame
data = []

# Iterate over each pain level directory
for pain_level in os.listdir(biosignals_path):
    pain_level_dir = os.path.join(biosignals_path, pain_level)
    
    # Check if it's a directory
    if os.path.isdir(pain_level_dir):
        # Iterate over each CSV file in the pain level directory
        for csv_file in os.listdir(pain_level_dir):
            # Check if it's a CSV file
            if csv_file.endswith('.csv'):
                csv_path = os.path.join(pain_level_dir, csv_file)
                # Read the CSV file
                df = pd.read_csv(csv_path, sep='\t')
                # Extract GSR values
                gsr_signal = df['gsr'].values
                # Append the CSV name, GSR signals, and Pain level to the data list
                data.append({'CSV name': csv_file, 'GSR signals': gsr_signal, 'Pain level': int(pain_level)})

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Display the DataFrame
df.head()


,CSV name,GSR signals,Pain level
0,072414_m_23-PA2-034_bio.csv,"[6.966839, 6.966161, 6.966, 6.966839, 6.966161...",2
1,081609_w_40-PA2-028_bio.csv,"[0.872, 0.872, 0.872, 0.872, 0.872, 0.872, 0.8...",2
2,081714_m_36-PA2-065_bio.csv,"[6.089862, 6.091, 6.091432, 6.092, 6.092432, 6...",2
3,102514_w_40-PA2-046_bio.csv,"[1.462, 1.462, 1.462, 1.462, 1.462, 1.462, 1.4...",2
4,120514_w_56-PA2-019_bio.csv,"[2.226, 2.226, 2.226, 2.226, 2.226, 2.226, 2.2...",2


In [68]:
print(df)

                         CSV name  \
0     072414_m_23-PA2-034_bio.csv   
1     081609_w_40-PA2-028_bio.csv   
2     081714_m_36-PA2-065_bio.csv   
3     102514_w_40-PA2-046_bio.csv   
4     120514_w_56-PA2-019_bio.csv   
...                           ...   
8695  082909_m_47-BL1-085_bio.csv   
8696  081609_w_40-BL1-090_bio.csv   
8697  091809_w_43-BL1-097_bio.csv   
8698  112016_m_25-BL1-091_bio.csv   
8699  083013_w_47-BL1-086_bio.csv   

                                            GSR signals  Pain level  
0     [6.966839, 6.966161, 6.966, 6.966839, 6.966161...           2  
1     [0.872, 0.872, 0.872, 0.872, 0.872, 0.872, 0.8...           2  
2     [6.089862, 6.091, 6.091432, 6.092, 6.092432, 6...           2  
3     [1.462, 1.462, 1.462, 1.462, 1.462, 1.462, 1.4...           2  
4     [2.226, 2.226, 2.226, 2.226, 2.226, 2.226, 2.2...           2  
...                                                 ...         ...  
8695  [3.184, 3.184399, 3.1846, 3.184, 3.184, 3.184,...          

### Selection of number of pain level included in the classification task

In [69]:
# Filter the DataFrame to keep rows where pain level is not equal to 1, 2, or 3
df = df[~df['Pain level'].isin([1, 2, 3])]

# Print the filtered DataFrame
print(df)

                         CSV name  \
5220  080314_w_25-PA4-067_bio.csv   
5221  092009_m_54-PA4-042_bio.csv   
5222  071709_w_23-PA4-071_bio.csv   
5223  082809_m_26-PA4-005_bio.csv   
5224  112909_w_20-PA4-080_bio.csv   
...                           ...   
8695  082909_m_47-BL1-085_bio.csv   
8696  081609_w_40-BL1-090_bio.csv   
8697  091809_w_43-BL1-097_bio.csv   
8698  112016_m_25-BL1-091_bio.csv   
8699  083013_w_47-BL1-086_bio.csv   

                                            GSR signals  Pain level  
5220  [4.521143, 4.522, 4.522, 4.522, 4.522, 4.522, ...           4  
5221  [1.673, 1.673, 1.673, 1.673, 1.673, 1.673, 1.6...           4  
5222  [4.326707, 4.327, 4.326292, 4.326708, 4.327, 4...           4  
5223  [3.913801, 3.913, 3.913, 3.913, 3.912801, 3.91...           4  
5224  [2.633, 2.633728, 2.634, 2.633272, 2.633728, 2...           4  
...                                                 ...         ...  
8695  [3.184, 3.184399, 3.1846, 3.184, 3.184, 3.184,...          

Preprocessing
================

Preprocessing steps for GSR DataFrame include tasks such as handling missing values, smoothing the signal to reduce noise in the GSR signal 9(Savitzky-Golay filtering), removing outliers (z-score), and normalizing the data between a specified range, such as [0, 1] or [-1, 1] helping comparison across different subjects.

In [70]:
# Function to preprocess GSR signals
def preprocess_gsr_signal(gsr_signal):
    # Handle missing values (if any)
    gsr_signal = np.array(gsr_signal)  # Convert to NumPy array
    gsr_signal = gsr_signal[~np.isnan(gsr_signal)]  # Remove NaN values
    
    # Check if the length of the signal is sufficient for smoothing
    if len(gsr_signal) < 5:
        # If the signal is too short, return the original signal
        return gsr_signal
    
    try:
        # Smoothing using Savitzky-Golay filter
        gsr_signal_smooth = savgol_filter(gsr_signal, window_length=5, polyorder=2)
    except ValueError:
        # If an error occurs during smoothing, return the original signal
        return gsr_signal
    
    # Removing outliers based on Z-scores
    z_scores = (gsr_signal_smooth - gsr_signal_smooth.mean()) / gsr_signal_smooth.std()
    gsr_signal_smooth_no_outliers = gsr_signal_smooth[(z_scores < 3)]
    
    # Normalization
    if len(gsr_signal_smooth_no_outliers) > 0:
        gsr_signal_normalized = (gsr_signal_smooth_no_outliers - gsr_signal_smooth_no_outliers.min()) / \
                                 (gsr_signal_smooth_no_outliers.max() - gsr_signal_smooth_no_outliers.min())
    else:
        # If there are no valid values after removing outliers, return the original signal
        return gsr_signal
    
    return gsr_signal_normalized

# Apply preprocessing to each row in the DataFrame
df['GSR signals'] = df['GSR signals'].apply(preprocess_gsr_signal)

# Display the updated DataFrame
df.head()

,CSV name,GSR signals,Pain level
5220,080314_w_25-PA4-067_bio.csv,"[0.0, 0.000784955924601847, 0.0012131137016623...",4
5221,092009_m_54-PA4-042_bio.csv,"[0.21654547886192313, 0.21654547886192313, 0.2...",4
5222,071709_w_23-PA4-071_bio.csv,"[1.0, 0.9983642739356123, 0.9979709555643436, ...",4
5223,082809_m_26-PA4-005_bio.csv,"[0.3629402970779174, 0.36178416384520784, 0.36...",4
5224,112909_w_20-PA4-080_bio.csv,"[0.16994520796566095, 0.1741950993228739, 0.17...",4


In [71]:
# Step 1: Prepare the data
max_length = max(len(signal) for signal in df['GSR signals'])  # Find the maximum length of GSR signals

# Pad or truncate the GSR signals to the maximum length
gsr_signals = np.array([np.pad(signal, (0, max_length - len(signal))) if len(signal) < max_length else signal[:max_length] for signal in df['GSR signals']])

pain_levels = df['Pain level'].values

# Step 2: Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(gsr_signals, pain_levels, test_size=0.2, random_state=42)

# Step 3: Convert the data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Check the shape of tensors
print("X_train_tensor shape:", X_train_tensor.shape)
print("X_test_tensor shape:", X_test_tensor.shape)
print("y_train_tensor shape:", y_train_tensor.shape)
print("y_test_tensor shape:", y_test_tensor.shape)


X_train_tensor shape: torch.Size([2784, 1, 2816])
X_test_tensor shape: torch.Size([696, 1, 2816])
y_train_tensor shape: torch.Size([2784])
y_test_tensor shape: torch.Size([696])


In [78]:
#check dimensionality of the tensor
inputs.shape

torch.Size([1024, 1, 2816])

The first dimension (29) represents the batch size, indicating that there are 29 samples in the batch.  
The second dimension (1) represents the number of channels. In this case, there is only one channel.  
The third dimension (1) represents the length of the input data for each channel.

Input data has a shape of (batch_size, channels, sequence_length).

1D- CNN model
================

I build and train a Convolutional Neural Network (CNN) using PyTorch for a classification task of pain levels. It defines the model architecture, sets up training parameters, prepares data tensors, performs K-Fold cross-validation, trains the model, evaluates it on validation sets, and finally evaluates its performance on a test set. 

It follows thoses steps :

1) **Define the CNN Model (Conv1D_model):**

This model consists of two convolutional layers followed by max-pooling layers and two fully connected layers. The first convolutional layer (conv1) has 32 output channels, a kernel size of 5, and a stride of 2. After each convolutional layer, a Rectified Linear Unit (ReLU) activation function is applied (relu1, relu2). Max-pooling layers (maxpool1, maxpool2) with kernel size 2 are used to reduce the spatial dimensions. The output of the convolutional layers is flattened and passed through two fully connected layers (fc1, fc2) for classification. The output layer does not have an activation function (sigmoid) because nn.CrossEntropyLoss already includes a softmax layer.

2) **Define Learning Parameters:**

Learning rate (lr), number of epochs (num_epochs), batch size (batch_size), and number of classes (num_classes) are defined.

3) **Define Loss Function and Optimizer:**

Cross-entropy loss (nn.CrossEntropyLoss()) is used as the loss function.
Stochastic Gradient Descent (SGD) optimizer (optim.SGD) with momentum is used for optimization.

4) **Prepare Training and Testing Data:**

Synthetic data tensors (X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor) are created. Replace them with your actual data.

5) **Perform K-Fold Cross-Validation:**

K-Fold cross-validation with 5 folds (k_folds) is performed using KFold from sklearn.model_selection.
Data is split into training and validation sets for each fold.
Training and validation data are loaded into PyTorch DataLoader objects.

6) **Train the Model:**

The model is trained for the specified number of epochs.
Training is done on the training data using mini-batches obtained from the train_loader.
After each epoch, the model is evaluated on the validation set to monitor performance.

7) **Evaluate the Model on Test Data:**

After cross-validation, the final trained model is evaluated on the test set to assess its generalization performance.

In [80]:
# Define the Conv1D model
class Conv1D_model(nn.Module):
    def __init__(self, num_classes=2):
        super(Conv1D_model, self).__init__()
        # First Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=2)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        
        # Second Convolutional Layer
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(22336, 512)  
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = x.view(x.size(0), -1)  # Flatten the tensor to 1D
        x = self.fc1(x)
        
        x = self.fc2(x)
        #x = self.sigmoid(x)
        return x

# Define learning parameters
lr = 0.0001
num_epochs = 10
batch_size = 1024
num_classes = 2

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Define training and testing tensors 

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  
y_train_tensor = torch.LongTensor(y_train)  
y_test_tensor = torch.LongTensor(y_test)    


# Initialize K-Fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train_tensor)):
    print(f"Fold {fold + 1}/{k_folds}")

    # Split data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train_tensor[train_index], X_train_tensor[val_index]
    y_train_fold, y_val_fold = y_train_tensor[train_index], y_train_tensor[val_index]

    # Create DataLoader for training and validation sets
    train_dataset = TensorDataset(X_train_fold, y_train_fold)
    val_dataset = TensorDataset(X_val_fold, y_val_fold)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    model = Conv1D_model(num_classes=num_classes)

    # Initialize the optimizer
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    # Evaluate the model on the validation set
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Validation Accuracy: {accuracy:.4f}")

Fold 1/5


IndexError: Target 4 is out of bounds.

Evaluation
============

In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = torch.argmax(outputs, dim=1)
    y_test_tensor_sampled = y_test_tensor[:len(predictions)]
    accuracy = torch.mean((predictions == y_test_tensor_sampled).float()).item()
    print(f"Test Accuracy: {accuracy:.4f}")

    # Calculate MAE and RMSE
    mae = mean_absolute_error(y_test_tensor_sampled, predictions)
    rmse = mean_squared_error(y_test_tensor_sampled, predictions, squared=False)
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")

Test Accuracy: 0.4822
MAE: 0.5178
RMSE: 0.7196


/home/ens/AU59350/anaconda3/envs/ClaraEnv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
